Deep Learning Course - Exercise 2
This is a softmax prototype, with classes: Happy, Sad, Angry
At first we tried using logistic regression with 2 classes: Happy, Sad
This didn't work well enough, adn we got ~0.5 accuracy.
We tried the same method but to predict Male or Female, and this worked much better with ~0.8 accuracy.
Eventually we tried doing softmax with Happy, Sad, Angry, to see if the problem was a weak model for happy and sad.

In [307]:
import tensorflow as tf
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import timeit

In [308]:
# Load the filenames from the folder containing the dataset
filenames = []
for file in os.listdir('audio2'):
    filenames.append(file)
print("num of files=",len(filenames),",num of actors=",len(filenames)/60)

num of files= 600 ,num of actors= 10.0


In [309]:
random.shuffle(filenames)

In [310]:
# max, min = 76117, 53310
# start, end = 21000/sampling_rate, 47424/sampling_rate
# wave_length = end-start
num_train = int(len(filenames)*0.7)
num_test = len(filenames) - num_train

In [311]:
# for filename in filenames[:num_train]:
#     data, sampling_rate = librosa.load("audio/" + filename, res_type='kaiser_fast', duration=end, offset=start)
#     print(len(data))

In [312]:
# df = pd.DataFrame(columns=['feature'])
# for index, y in enumerate(filenames):
#     data, sampling_rate = librosa.load("audio/" + filename, res_type='kaiser_fast', duration=end, offset=start)
#     mfccs = np.mean(librosa.feature.mfcc(y=data, n_mfcc=25,), axis=0)
#     data_x_train.append(-(mfccs)/100)
    
# df

In [313]:
data_x_train = []
data_x_test = []

start_time = timeit.default_timer()
for filename in filenames[:num_train]:
    data, sampling_rate = librosa.load("audio2/" + filename, sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
    sampling_rate = np.array(sampling_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
    data_x_train.append(mfccs)
    
for filename in filenames[num_train:]:
    data, sampling_rate = librosa.load("audio2/" + filename, sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
    sampling_rate = np.array(sampling_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
    data_x_test.append(mfccs)
    
stop_time = timeit.default_timer()
print('runtime: ', stop_time - start_time)  

runtime:  35.29020342662443


In [314]:
data_y_train = []
data_y_test = []

for filename in filenames[:num_train]:
    if filename[7] == '1':
        data_y_train.append([1,0,0,0,0,0,0,0])
    elif filename[7] == '2':
        data_y_train.append([0,1,0,0,0,0,0,0])
    elif filename[7] == '3':
        data_y_train.append([0,0,1,0,0,0,0,0])
    elif filename[7] == '4':
        data_y_train.append([0,0,0,1,0,0,0,0])
    elif filename[7] == '5':
        data_y_train.append([0,0,0,0,1,0,0,0])
    elif filename[7] == '6':
        data_y_train.append([0,0,0,0,0,1,0,0])
    elif filename[7] == '7':
        data_y_train.append([0,0,0,0,0,0,1,0])
    elif filename[7] == '8':
        data_y_train.append([0,0,0,0,0,0,0,1])
        
for filename in filenames[num_train:]:
    if filename[7] == '1':
        data_y_test.append([1,0,0,0,0,0,0,0])
    elif filename[7] == '2':
        data_y_test.append([0,1,0,0,0,0,0,0])
    elif filename[7] == '3':
        data_y_test.append([0,0,1,0,0,0,0,0])
    elif filename[7] == '4':
        data_y_test.append([0,0,0,1,0,0,0,0])
    elif filename[7] == '5':
        data_y_test.append([0,0,0,0,1,0,0,0])
    elif filename[7] == '6':
        data_y_test.append([0,0,0,0,0,1,0,0])
    elif filename[7] == '7':
        data_y_test.append([0,0,0,0,0,0,1,0])
    elif filename[7] == '8':
        data_y_test.append([0,0,0,0,0,0,0,1])

In [315]:
# takes every value in data_y to an array
# ex: [0, 1, 1, 1, 0] => [[0], [1], [1], [1], [0]]
data_y_train_correct = []
data_y_test_correct = []

for val in data_y_train:
    val_arr = []
    val_arr.append(val)
    data_y_train_correct.append(val_arr)
    
for val in data_y_test:
    val_arr = []
    val_arr.append(val)
    data_y_test_correct.append(val_arr)

In [316]:
data_xy_train = []
for i in range(len(data_x_train)):
    data_xy_train.append( (data_x_train[i], data_y_train[i]) )
    
data_xy_test = []
for i in range(len(data_x_test)):
    data_xy_test.append( (data_x_test[i], data_y_test[i]) )

In [317]:
def getXvalues(data_xy):
    x_values = []
    for data in data_xy:
        x_values.append(data[0])
    return x_values

def getYvalues(data_xy):
    y_values = []
    for data in data_xy:
        y_values.append(data[1])
    return y_values

In [318]:
def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

In [353]:
0.33333334
features = len(data_xy_train[0][0])

x = tf.placeholder(tf.float32, [None, features])
y_ = tf.placeholder(tf.float32, [None, 8])
W = tf.Variable(tf.zeros([features, 8]))
b = tf.Variable(tf.zeros([8]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1])+0.1*tf.nn.l2_loss(W))
train_step = tf.train.GradientDescentOptimizer(0.00002).minimize(cross_entropy)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

start_time = timeit.default_timer()

for i in range(950):
    sess.run(train_step, feed_dict={x:getXvalues(data_xy_train), y_:getYvalues(data_xy_train)})
#     random.shuffle(data_xy_train)
    
stop_time = timeit.default_timer()
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x:getXvalues(data_xy_test), y_:getYvalues(data_xy_test)}))
print('runtime: ', stop_time - start_time)  

0.33333334
runtime:  3.0462227212083235


In [335]:
file_test = []

data, sampling_rate = librosa.load("yosi3.wav", sr=22050*2, res_type='kaiser_fast', duration=2.5, offset=0.5)
sampling_rate = np.array(sampling_rate)
mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=13), axis=0)
file_test.append(mfccs)

In [336]:
sess.run(tf.argmax(y,1)+1, feed_dict={x:file_test})


array([7], dtype=int64)